In [30]:
import cv2
import torch
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
import pandas as pd
import pytorch_lightning as pl
from torchvision.io import read_image
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torchvision.transforms import ToTensor
from PIL import Image

## Load the data

In [31]:
test_path = './data/chest_xray/'
train_path = './data/chest_xray/'
val_path = './data/chest_xray/'

In [32]:
img_info = pd.read_csv('./img_info.csv')
test_df = img_info.loc[img_info.loc[:, 'data'] == 'Test']
train_df = img_info.loc[img_info.loc[:, 'data'] == 'Train']
val_df = test_df = img_info.loc[img_info.loc[:, 'data'] == 'Validation']

In [33]:
normal_images = train_df.loc[train_df.loc[:,'label'] == 0]
normal_images.loc[:,'imbalance'] = 1
train_df = pd.concat([train_df, normal_images, normal_images]).reset_index(drop=True)
print('Percentage of Normal Images in the Training Data: {}'.format(
    len(train_df.loc[train_df.loc[:,'label'] == 0])/len(train_df.loc[:,'label'])))
print('Percentage of Pneumonia Images in the Training Data: {}'.format(
    len(train_df.loc[train_df.loc[:,'label'] == 1])/len(train_df.loc[:,'label'])))

Percentage of Normal Images in the Training Data: 0.5093694606229425
Percentage of Pneumonia Images in the Training Data: 0.4906305393770575


/var/folders/rs/dwj3n8716t593gn_r4x21y0r0000gn/T/ipykernel_4697/907957275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_images.loc[:,'imbalance'] = 1


In [34]:
class CustomImageDataset(Dataset):
    def __init__(self, df, img_dir_path, transforms=None):
        """
        You can set your custom dataset to take in more parameters than specified
        here. But, I recommend at least you start with the three I listed here,
        as these are standard

        csv_file (str): file path to the csv file you created /
        df (pandas df): pandas dataframe

        img_dir_path: directory path to your images
        transform: Compose (a PyTorch Class) that strings together several
          transform functions (e.g. data augmentation steps)

        One thing to note -- you technically could implement `transform` within
        the dataset. No one is going to stop you, but you can think of the
        transformations/augmentations you do as a hyperparameter. If you treat
        it as a hyperparameter, you want to be able to experiment with different
        transformations, and therefore, it would make more sense to decide those
        transformations outside the dataset class and pass it to the dataset!
        """
        self.img_labels = df
        self.img_dir = img_dir_path
        self.transforms = transforms
        
    def __len__(self):
        """
        Returns: (int) length of your dataset
        """
        return len(self.img_labels)

    def __getitem__(self, idx):
        """
        Loads and returns your sample (the image and the label) at the
        specified index

        Parameter: idx (int): index of interest

        Returns: image, label
        """

        img_path =  self.img_dir + self.img_labels.iloc[idx, 1]
        
        image = Image.open(img_path).convert('RGB')

        label = self.img_labels.iloc[idx, -1]
        
        imbalance = self.img_labels.iloc[idx, -2]

        if self.transforms:
            
            if imbalance and not label:
                image = transforms(image)
                image = imbalance_transform(image)
                
            else:
                image = transforms(image)
               
        return image, label

In [35]:
transforms = T.Compose(
    [
        T.Resize((224,224), antialias=None, interpolation=InterpolationMode.BICUBIC),
        T.RandomApply([
            T.GaussianBlur(kernel_size=(5,5), sigma=(0.1, 0.2))
        ], p=0.5),
        T.RandomEqualize(),
        T.ToTensor()
    ]
)


imbalance_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    T.RandomErasing(p=0.2)
])

In [36]:
training_data = CustomImageDataset(train_df, train_path, transforms=transforms)
val_data = CustomImageDataset(val_df, val_path, transforms=transforms)
test_data = CustomImageDataset(test_df, test_path, transforms=transforms)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [37]:
training_data[0]

(tensor([[[0.0784, 0.0941, 0.0941,  ..., 0.0902, 0.0980, 0.0275],
          [0.0784, 0.0941, 0.0941,  ..., 0.0941, 0.0941, 0.0275],
          [0.0745, 0.0941, 0.0980,  ..., 0.0941, 0.0941, 0.0275],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0118, 0.0118],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0784, 0.0941, 0.0941,  ..., 0.0902, 0.0980, 0.0275],
          [0.0784, 0.0941, 0.0941,  ..., 0.0941, 0.0941, 0.0275],
          [0.0745, 0.0941, 0.0980,  ..., 0.0941, 0.0941, 0.0275],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0118, 0.0118],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0784, 0.0941, 0.0941,  ..., 0.0902, 0.0980, 0.0275],
          [0.0784, 0.0941, 0.0941,  ..., 0.0941, 0.0941, 0.0275],
          [0.0745, 0.0941, 0.0980,  ...,

## Model Setup

In [38]:
class CustomNeuralNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        
        self.LeNet = nn.Sequential(     
            # convolutional layers            
            nn.Sequential(                                            # FIRST LAYER: (INPUT LAYER)
              nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=0),    # CONVOLUTION 
              nn.BatchNorm2d(6),
              nn.Sigmoid(),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            nn.Sequential(                                            # SECOND LAYER: HIDDEN LAYER 1
              nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=0),   # CONVOLUTION 
              nn.BatchNorm2d(16),
              nn.Sigmoid(),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            # fully connected layers
            nn.Flatten(),
            # output layer
            nn.Linear(16 * 54 * 54, 2)                                # OUTPUT LAYER
        )

    def forward(self, x):
        out = self.LeNet(x)
        
        return out


In [39]:
model_1 = CustomNeuralNetwork()

In [40]:
# Define a Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_1.parameters(), lr=0.001, momentum=0.9)

In [41]:
import math

In [29]:
EPOCHS = 20

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(EPOCHS):  # loop over the dataset multiple times

    # TRAIN
    # Make sure gradient tracking is on, and do a pass over the data
    model_1.train()
    running_loss = 0.0
    running_matched = 0
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data           # NOTE: depending on how you implemented your dataset class's __getitem__ it could be labels, inputs

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_1.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # keep track of the loss
        running_loss += loss.item()

        # CALCULATE ACCURACY METRIC
        _, preds = torch.max(outputs, 1)  # Find out the predicted class with the highest prob
        running_matched += torch.sum(preds == labels.data) # caculate the number of matched labels

    avg_train_loss = running_loss / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
    avg_train_acc = running_matched.double() / len(train_dataloader.dataset)
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)

    # VALIDATE
    # In the validation part, we don't want to keep track of the gradients 
    model_1.eval()
    running_val_loss = 0.0
    running_val_matched = 0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            inputs, labels = data
            outputs = model_1(inputs)
            loss = criterion(outputs, labels)

            # keep track of the loss
            running_val_loss += loss.item()

            # CALCULATE ACCURACY METRIC
            _, preds = torch.max(outputs, 1)
            running_val_matched += torch.sum(preds == labels.data)

    avg_val_loss = running_val_loss / (i + 1)
    avg_val_acc = running_val_matched.double() / len(val_dataloader.dataset)
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_acc)

    # Print epoch summary
    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")


KeyboardInterrupt: 

In [ ]:
PATH = '/Users/miaoli/Desktop/Uchi/assignments/spring_2023/machine learning/project/sigmoid.pth'
torch.save(model_1.state_dict(), PATH)

In [ ]:
train_acc = []
for x in train_accuracies:
    train_acc.append(x.tolist())

val_acc = []
for x in val_accuracies:
    val_acc.append(x.tolist())

In [ ]:
result = {}
result['train_losses'] = train_losses
result['train_accuracies'] = train_acc
result['val_losses'] =val_losses
result['val_accuracies'] = val_acc